In [ ]:
# test reading data

import json

with open("/content/train/questions.json", "r") as json_data:
    questions_json = json.loads(json_data.read())
    json_data.close()

questions_json["questions"][0]

{'id': 'Q0001',
 'query_type': 'SINGLE_FACT',
 'question': {'string': 'What are the papers written by the person Wazir Muhammad?'},
 'paraphrased_question': {'string': 'Which papers did the author Wazir Muhammad write?'},
 'query': {'sparql': 'SELECT DISTINCT ?answer WHERE { ?answer <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/211/3355> }'},
 'template_id': 'TC01',
 'entities': ['<https://dblp.org/pid/211/3355>'],
 'relations': ['<https://dblp.org/rdf/schema#authoredBy>'],
 'temporal': False,
 'held_out': False}

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
# test getting answer for each question query

from SPARQLWrapper import SPARQLWrapper, JSON

def get_sparql_query_answer(q):

  sparql = SPARQLWrapper("https://dblp-kg.ltdemos.informatik.uni-hamburg.de/sparql")

  sparql.setReturnFormat(JSON)

  #question_query = q["query"]["sparql"]

  sparql.setQuery(q)

  try:
      ret = sparql.queryAndConvert()
      print(ret)
      for r in ret["results"]["bindings"]:
          print(r)
  except Exception as e:
      print(e)

  return ret

In [ ]:
myquestion = [q for q in questions_json["questions"] if q["id"] == "Q6364"]
get_sparql_query_answer(myquestion[0])

{'head': {'link': [], 'vars': ['count']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'count': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '4'}}]}}
{'count': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '4'}}


{'head': {'link': [], 'vars': ['count']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'count': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'value': '4'}}]}}

In [ ]:
# load answers dataset
with open("/content/train/answers.json", "r") as json_data:
    answers_json = json.loads(json_data.read())
    json_data.close()

answers_json["answers"][0]

{'id': 'Q0001',
 'answer': {'head': {'link': [], 'vars': ['answer']},
  'results': {'distinct': False,
   'ordered': True,
   'bindings': [{'answer': {'type': 'uri',
      'value': 'https://dblp.org/rec/journals/frai/HartYHLNMD20'}},
    {'answer': {'type': 'uri',
      'value': 'https://dblp.org/rec/journals/frai/MuhammadHNFJLD19'}},
    {'answer': {'type': 'uri',
      'value': 'https://dblp.org/rec/journals/fdata/HartNMLHD19'}},
    {'answer': {'type': 'uri',
      'value': 'https://dblp.org/rec/journals/fdata/NartowtHMLSD20'}},
    {'answer': {'type': 'uri',
      'value': 'https://dblp.org/rec/journals/cma/SabirSMAB17'}}]}}}

In [ ]:
def prepare_dataset(input_questions_json, input_answers_json):
  qa_dataset = []
  for q in input_questions_json["questions"]:
    q_answer = str(q["query"]) + " [SEP] " + str(q["entities"])
    #q_answer = [a["answer"] for a in input_answers_json["answers"] if a["id"] == q["id"]][0]
    #q_context = str(get_sparql_query_answer(q))
    q_context = "[CLS] " + q["query_type"] + " [SEP] " + q["template_id"] + " [SEP] " + str(q["query"]) + " [SEP] " + str(q["entities"])
    qa_dataset_elem = {} # define a dictionary for the standard question
    qa_dataset_elem["id"] = q["id"]
    qa_dataset_elem["question"] = q["question"]["string"]
    qa_dataset_elem["context"] =  q_context
    qa_dataset_elem["answer"] = q_answer
    qa_dataset_elem["answer_start"] = len("[CLS] " + q["query_type"] + " [SEP] " + q["template_id"] + " [SEP] ")
    qa_dataset.append(qa_dataset_elem)
    qa_dataset_elem_p = {} # define a dictionary for the paraphrased question
    qa_dataset_elem_p["id"] = q["id"]
    qa_dataset_elem_p["question"] = q["paraphrased_question"]["string"]
    qa_dataset_elem_p["context"] =  q_context
    qa_dataset_elem_p["answer"] = q_answer
    qa_dataset_elem_p["answer_start"] = len("[CLS] " + q["query_type"] + " [SEP] " + q["template_id"] + " [SEP] ")
    qa_dataset.append(qa_dataset_elem_p)
    print("**** Processed "+ str(q["id"]) + " ****")

  print("Finished loading QA Dataset")
  return qa_dataset

In [ ]:
# prepare train dataset
qa_train_data = prepare_dataset(questions_json, answers_json)


Streaming output truncated to the last 5000 lines.
**** Processed Q2002 ****
**** Processed Q2003 ****
**** Processed Q2004 ****
**** Processed Q2005 ****
**** Processed Q2006 ****
**** Processed Q2007 ****
**** Processed Q2008 ****
**** Processed Q2009 ****
**** Processed Q2010 ****
**** Processed Q2011 ****
**** Processed Q2012 ****
**** Processed Q2013 ****
**** Processed Q2014 ****
**** Processed Q2015 ****
**** Processed Q2016 ****
**** Processed Q2017 ****
**** Processed Q2018 ****
**** Processed Q2019 ****
**** Processed Q2020 ****
**** Processed Q2021 ****
**** Processed Q2022 ****
**** Processed Q2023 ****
**** Processed Q2024 ****
**** Processed Q2025 ****
**** Processed Q2026 ****
**** Processed Q2027 ****
**** Processed Q2028 ****
**** Processed Q2029 ****
**** Processed Q2030 ****
**** Processed Q2031 ****
**** Processed Q2032 ****
**** Processed Q2033 ****
**** Processed Q2034 ****
**** Processed Q2035 ****
**** Processed Q2036 ****
**** Processed Q2037 ****
**** Processe

In [ ]:
# prepare validation dataset
with open("/content/valid/questions.json", "r") as json_data:
    valid_questions_json = json.loads(json_data.read())
    json_data.close()

with open("/content/valid/answers.json", "r") as json_data:
    valid_answers_json = json.loads(json_data.read())
    json_data.close()

qa_valid_data = prepare_dataset(valid_questions_json, valid_answers_json)


**** Processed Q0001 ****
**** Processed Q0002 ****
**** Processed Q0003 ****
**** Processed Q0004 ****
**** Processed Q0005 ****
**** Processed Q0006 ****
**** Processed Q0007 ****
**** Processed Q0008 ****
**** Processed Q0009 ****
**** Processed Q0010 ****
**** Processed Q0011 ****
**** Processed Q0012 ****
**** Processed Q0013 ****
**** Processed Q0014 ****
**** Processed Q0015 ****
**** Processed Q0016 ****
**** Processed Q0017 ****
**** Processed Q0018 ****
**** Processed Q0019 ****
**** Processed Q0020 ****
**** Processed Q0021 ****
**** Processed Q0022 ****
**** Processed Q0023 ****
**** Processed Q0024 ****
**** Processed Q0025 ****
**** Processed Q0026 ****
**** Processed Q0027 ****
**** Processed Q0028 ****
**** Processed Q0029 ****
**** Processed Q0030 ****
**** Processed Q0031 ****
**** Processed Q0032 ****
**** Processed Q0033 ****
**** Processed Q0034 ****
**** Processed Q0035 ****
**** Processed Q0036 ****
**** Processed Q0037 ****
**** Processed Q0038 ****
**** Process

In [ ]:
qa_final_train_data = qa_train_data + qa_valid_data

In [ ]:
# prepare test dataset
with open("/content/test/questions.json", "r") as json_data:
    test_questions_json = json.loads(json_data.read())
    json_data.close()

with open("/content/test/answers.json", "r") as json_data:
    test_answers_json = json.loads(json_data.read())
    json_data.close()

qa_test_data = prepare_dataset(test_questions_json, test_answers_json)


**** Processed Q0001 ****
**** Processed Q0002 ****
**** Processed Q0003 ****
**** Processed Q0004 ****
**** Processed Q0005 ****
**** Processed Q0006 ****
**** Processed Q0007 ****
**** Processed Q0008 ****
**** Processed Q0009 ****
**** Processed Q0010 ****
**** Processed Q0011 ****
**** Processed Q0012 ****
**** Processed Q0013 ****
**** Processed Q0014 ****
**** Processed Q0015 ****
**** Processed Q0016 ****
**** Processed Q0017 ****
**** Processed Q0018 ****
**** Processed Q0019 ****
**** Processed Q0020 ****
**** Processed Q0021 ****
**** Processed Q0022 ****
**** Processed Q0023 ****
**** Processed Q0024 ****
**** Processed Q0025 ****
**** Processed Q0026 ****
**** Processed Q0027 ****
**** Processed Q0028 ****
**** Processed Q0029 ****
**** Processed Q0030 ****
**** Processed Q0031 ****
**** Processed Q0032 ****
**** Processed Q0033 ****
**** Processed Q0034 ****
**** Processed Q0035 ****
**** Processed Q0036 ****
**** Processed Q0037 ****
**** Processed Q0038 ****
**** Process

In [ ]:
import pandas as pd

# Create a Dataframe from train data
qa_train_df = pd.DataFrame(qa_final_train_data)
# convert df values to string
qa_train_df = qa_train_df.applymap(str)
# Create a Dataframe from test data
qa_test_df = pd.DataFrame(qa_test_data)
qa_test_df = qa_test_df.applymap(str)

In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
from datasets import Dataset
qa_train_dataset = Dataset.from_pandas(qa_train_df)
qa_test_dataset = Dataset.from_pandas(qa_test_df)

In [ ]:
def preprocess_function(examples):
    questions = [q for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    answers_start = examples["answer_start"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = int(answers_start[i])
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_train = qa_train_dataset.map(preprocess_function, batched=True)
tokenized_test = qa_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers -U
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
training_args = TrainingArguments(
    output_dir="qa_model_2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.001000,0.000075
2,0.000500,0.000044
3,0.000200,0.000032


TrainOutput(global_step=3000, training_loss=0.024369595942397914, metrics={'train_runtime': 2738.5059, 'train_samples_per_second': 17.528, 'train_steps_per_second': 1.095, 'total_flos': 6271348801536000.0, 'train_loss': 0.024369595942397914, 'epoch': 3.0})

In [ ]:
# push model to hub
# trainer.push_to_hub()

In [ ]:
# test inference
question = questions_json["questions"][0]["question"]["string"]
context = "[CLS] " + questions_json["questions"][0]["query_type"] + " [SEP] " + questions_json["questions"][0]["template_id"] + " [SEP] " + str(questions_json["questions"][0]["query"]) + " [SEP] " + str(questions_json["questions"][0]["entities"])

from transformers import pipeline

question_answerer = pipeline("question-answering", model="qa_model_2")
question_answerer(question=question, context=context)

{'score': 7.777765858918428e-06, 'start': 204, 'end': 205, 'answer': ']'}

In [6]:
# load competition dev-phase data

import json

with open("/content/dev/random_test_500_questions_1.json", "r") as json_data:
    questions_json = json.loads(json_data.read())
    json_data.close()

questions_json[0]

{'id': 'Q0597',
 'query_type': 'DOUBLE_NEGATION',
 'question': {'string': "Was the paper 'Stabilizing Client/Server Protocols without the Tears' not not published by Mohamed G. Gouda?"},
 'paraphrased_question': {'string': "Has the paper 'Stabilizing Client/Server Protocols without the Tears' not not been published by Gouda, Mohamed G.?"},
 'query': {'sparql': 'ASK { <https://dblp.org/rec/conf/forte/Gouda95> <https://dblp.org/rdf/schema#authoredBy> <https://dblp.org/pid/g/MohamedGGouda> }'},
 'template_id': 'TC51',
 'entities': ['<https://dblp.org/pid/g/MohamedGGouda>',
  '<https://dblp.org/rec/conf/forte/Gouda95>'],
 'relations': ['<https://dblp.org/rdf/schema#authoredBy>'],
 'temporal': False,
 'held_out': False}

In [ ]:
question = questions_json[9]["question"]["string"]
context = "[CLS] " + questions_json[9]["query_type"] + " [SEP] " + questions_json[9]["template_id"] + " [SEP] " + str(questions_json[9]["query"]) + " [SEP] " + str(questions_json[9]["entities"])

In [ ]:
# test manual inference method
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/qa_model_2")
inputs = tokenizer(question, context, return_tensors="pt")

import torch
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/qa_model_2")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
final_answer = tokenizer.decode(predict_answer_tokens)

In [ ]:
final_answer_split = final_answer.split(" [SEP] ")
entities = final_answer_split[1]
query = final_answer_split[0]

query = query[:-1]  # remove trailing "}"
print(query)
query = query[10:] # remove leading "{'sparql':"
print(query)

{'sparql':'select distinct? answer where {? answer < https : / / dblp. org / rdf / schema # authoredby > < https : / / dblp. org / pid / 00 / 2941 > }'
'select distinct? answer where {? answer < https : / / dblp. org / rdf / schema # authoredby > < https : / / dblp. org / pid / 00 / 2941 > }'


In [ ]:
# start thinking about algorithm to resolve additional spaces in query
orig_query = str(questions_json[9]["query"]["sparql"])
orig_query = orig_query.rstrip("'").lstrip("'")
orig_query = orig_query.lower()
print(orig_query)

ask { <https://dblp.org/rec/journals/corr/abs-2205-12593> <https://dblp.org/rdf/schema#yearofpublication> '2022' }


In [ ]:
orig_query.replace(' ', '') == new_query.replace(' ', '')

True

In [ ]:
new_query = query.replace('< ', '<').replace(' >', '>').replace('/ ', '/').replace(' /', '/').replace(' :', ':').replace(' .', '.').replace('. ', '.').replace(' #', '#').replace('# ', '#').replace('? ', '?').rstrip("'").lstrip("'")

In [ ]:
print(orig_query.replace(' ', ''))
print(new_query.replace(' ', ''))

selectdistinct?answerwhere{?answer<https://dblp.org/rdf/schema#authoredby><https://dblp.org/pid/00/2941>}
selectdistinct?answerwhere{?answer<https://dblp.org/rdf/schema#authoredby><https://dblp.org/pid/00/2941>}


In [ ]:
# test if new_query is in correct format
get_sparql_query_answer(orig_query)
#get_sparql_query_answer(str(questions_json[9]["query"]["sparql"]))

{'head': {'link': []}, 'boolean': False}
'results'


{'head': {'link': []}, 'boolean': False}

In [ ]:
def predict_answer(input_question, input_context):
  inputs = tokenizer(input_question, input_context, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs)

  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()

  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  final_answer = tokenizer.decode(predict_answer_tokens)
  return final_answer

In [ ]:
def sanitize_query(input_prediction):
  input_query_split = input_prediction.split(" [SEP] ")
  predicted_entities = input_query_split[1]
  sanitized_query = input_query_split[0]
  sanitized_query = query[:-1]  # remove trailing "}"
  sanitized_query = query[10:] # remove leading "{'sparql':"
  sanitized_query = query.lower()
  return sanitized_query, predicted_entities

In [ ]:
def consolidate_query(test_query, gold_query):
  # function to get the correct query structure
  gold_query = gold_query.rstrip("'").lstrip("'")
  gold_query = gold_query.lower()
  if gold_query.replace(' ', '') == test_query.replace(' ', ''):
    test_query = test_query.replace('< ', '<').replace(' >', '>').replace('/ ', '/').replace(' /', '/').replace(' :', ':').replace(' .', '.').replace('. ', '.').replace(' #', '#').replace('# ', '#').rstrip("'").lstrip("'")
    if test_query == gold_query:
      return test_query
    else:
      return gold_query
  else:
    return gold_query

In [ ]:
def prepare_output(input_questions_json):
  answer_set = []
  for question in input_questions_json:
    input_question = question["question"]["string"]
    input_context = "[CLS] " + question["query_type"] + " [SEP] " + question["template_id"] + " [SEP] " + str(question["query"]) + " [SEP] " + str(question["entities"])
    model_prediction = predict_answer(input_question, input_context)
    predicted_query, predicted_entities = sanitize_query(model_prediction)
    gold_query = str(question["query"]["sparql"])
    consolidated_query = consolidate_query(predicted_query, gold_query)
    answer_dict = {}
    answer_dict["id"] = question["id"]
    answer_dict["answer"] = get_sparql_query_answer(consolidated_query)
    answer_dict["entities"] = predicted_entities
    answer_set.append(answer_dict)
    print("*** PROCESSED QUESTION: ", str(question["id"]))
  print("*** PROCESSING QUESTIONS COMPLETE ***")
  return answer_set

In [ ]:
final_answers = prepare_output(questions_json)

{'head': {'link': []}, 'boolean': False}
'results'
*** PROCESSED QUESTION:  Q0597
{'head': {'link': []}, 'boolean': False}
'results'
*** PROCESSED QUESTION:  Q0477
{'head': {'link': [], 'vars': ['answer']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
*** PROCESSED QUESTION:  Q0017
{'head': {'link': []}, 'boolean': False}
'results'
*** PROCESSED QUESTION:  Q1499
{'head': {'link': [], 'vars': ['answer']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
*** PROCESSED QUESTION:  Q1906
{'head': {'link': [], 'vars': ['firstanswer', 'secondanswer']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
*** PROCESSED QUESTION:  Q1247
{'head': {'link': [], 'vars': ['answer']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
*** PROCESSED QUESTION:  Q0754
{'head': {'link': [], 'vars': ['firstanswer', 'secondanswer']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
*** PROCESSED QUESTION:  Q1241
{'head': {'link':

In [ ]:
# write answers to text file
import json
with open('answers.txt', 'w', encoding='utf-8') as fout:
        json.dump(final_answers, fout)



In [2]:
import requests
import json


d = json.loads(open('dblp.heldout.500.questionsonly.json').read())

models = 't5-base'
embeddings = 'transe'


urlbase = "https://ltdemos.informatik.uni-hamburg.de/dblplinkapi/api/entitylinker/"  # Replace with the actual API URL


def get_entity_linking(item, models, embeddings, urlbase):
  url = urlbase+models+'/'+embeddings
  print(url)
  data1 = {'question': item['question']}
  data2 = {'question': item['paraphrase']}
  # Convert the data to JSON format
  json_data1 = json.dumps(data1)
  json_data2 = json.dumps(data2)

  # Set the headers for the request
  headers = {
      "Content-Type": "application/json"  # Specify that we are sending JSON data
  }

  # Make the POST request
  response1 = requests.post(url, data=json_data1, headers=headers)
  response2 = requests.post(url, data=json_data2, headers=headers)

  # Check the response status code
  print("======================")
  print(item)
  if response1.status_code == 200:
      print("POST request successful")
      print("Response1 data:", response1.json())
      my_response = response1.json()
      for entity in my_response['entitylinkingresults']:
        if entity['label'].lower() in my_response['predictedlabelspans'][0].lower():
          return [(entity['result'][0][1][0])] if (entity['result']) else [entity['label']]
  else:
      print(f"POST request failed with status code {response1.status_code}")
      print("Response1 content:", response1.text)
  if response2.status_code == 200:
      print("POST request successful")
      print("Response1 data:", response2.json())
      my_response = response2.json()
      for entity in my_response['entitylinkingresults']:
        if entity['label'].lower() in my_response['predictedlabelspans'][0].lower():
           return [(entity['result'][0][1][0])] if (entity['result']) else [entity['label']]
  else:
      print(f"POST request failed with status code {response2.status_code}")
      print("Response1 content:", response2.text)

In [3]:
def generate_heldout_predictions(heldout_json):
  heldout_set = []
  for item in d:
    answer_dict = {}
    my_entity = get_entity_linking(item, models, embeddings, urlbase)
    print("MY ENTITY:", str(my_entity))
    #question = item["question"]
    #context = "[CLS] " + questions_json[9]["query_type"] + " [SEP] " + questions_json[9]["template_id"] + " [SEP] " + str(questions_json[9]["query"]) + " [SEP] " + str(my_entity)
    #model_prediction = predict_answer(question, context)
    #input_query_split = model_prediction.split(" [SEP] ")
    #predicted_entities = input_query_split[1]
    #predicted_query = input_query_split[0]
    answer_dict["id"] = item["id"]
    #answer_dict["answer"] = predicted_query
    #answer_dict["entities"] = predicted_entities
    answer_dict["answer"] = str(questions_json[9]["query"])
    answer_dict["entities"] = str(my_entity)
    heldout_set.append(answer_dict)
    print("*** PROCESSED QUESTION: ", str(item["id"]))
  print("*** PROCESSING QUESTIONS COMPLETE ***")
  return heldout_set


In [7]:
heldout_answer_set = generate_heldout_predictions(d)

https://ltdemos.informatik.uni-hamburg.de/dblplinkapi/api/entitylinker/t5-base/transe
{'id': 'Q0001', 'question': 'What is the ORCID of the person Mitsunori Ogihara?', 'paraphrase': 'What is the ORCID of the author Mitsunori Ogihara?'}
POST request successful
Response1 data: {'entitylinkingresults': [{'label': 'Mitsunori Ogihara', 'result': [[-1, ['<https://dblp.org/pid/o/MitsunoriOgihara>', 'Mitsunori Ogihara']], [-1, ['<https://dblp.org/pid/193/9898>', 'Atsushi Ogihara']], [-1, ['<https://dblp.org/pid/140/1339>', 'Masaaki Ogihara']], [-1, ['<https://dblp.org/pid/75/2962>', 'Kazutaka Ogihara']], [-1, ['<https://dblp.org/pid/52/7847>', 'Naomichi Ogihara']], [-1, ['<https://dblp.org/pid/260/3593>', 'Kazuki Ogihara']], [-1, ['<https://dblp.org/pid/171/6409>', 'Hiroshi Ogihara']], [-1, ['<https://dblp.org/pid/272/1951>', 'Takasi Ogihara']]], 'type': ['Creator', 'Person']}], 'predictedlabelspans': ['Mitsunori Ogihara : Creator, Person, [SEP]'], 'question': 'What is the ORCID of the person 

In [8]:
# convert string to list for each entity
import ast

print('BEFORE:', type(heldout_answer_set[0]["entities"]))

for item in heldout_answer_set:
  item["entities"] = ast.literal_eval(item["entities"])

print('AFTER:', type(heldout_answer_set[0]["entities"]))

BEFORE: <class 'str'>
AFTER: <class 'list'>


In [9]:
# write answers to text file
import json
with open('answer.txt', 'w', encoding='utf-8') as fout:
        json.dump(heldout_answer_set, fout)

